# Librerias

In [2]:
!pip install scikit-learn

In [2]:
!pip install pandas

In [3]:
!pip install numpy

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import warnings
warnings.filterwarnings('ignore')

# Procedimiento

Crearemos una función para cada estilo definido distinguiendo en él las distintas demarcaciones de los jugadores.

In [1]:
import pandas as pd
import numpy as np

class EstiloFutbolJugadores(object):
    """
    Clase que devuelve el TOP de futbolistas atendiendo a puntuaciones en ataque y defensa, y un score final.
    """
    
    def __init__(self, file):
        """
        Inicializa con el fichero Excel.
        """
        self.file = file

    def read_preprocess(self):
        """
        Lee el fichero y normaliza las métricas (entre 0 y 1).
        """
        data = pd.read_excel(self.file)

        # Nueva variable creada por el usuario
        data['G-xG/90'] = data['Gls/90'] - data['xG/90']

        # Normalización manual
        data_scaler = data.iloc[:, 6:].copy()
        for col in data_scaler.columns:
            min_val = data_scaler[col].min()
            max_val = data_scaler[col].max()
            if max_val != min_val:
                data_scaler[col] = (data_scaler[col] - min_val) / (max_val - min_val)
            else:
                data_scaler[col] = 0

        df_scaler = pd.concat([data.iloc[:, :6], data_scaler], axis=1)
        return df_scaler, data

    def multiply(self, pcts_ataque, pcts_defensa, cols):
        """
        Calcula puntuación de ataque y defensa.
        """
        value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
        value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
        return value_ataque, value_defensa

    def algorithm(self, demarcaciones:list, cols:list, percentile_min:int, n_top:int, is_ascending:bool, 
                  list_pct_ataque:list, list_pct_defensa:list):
        """
        Ejecuta el algoritmo para calcular puntuaciones.
        """
        df_scaled, df_original = self.read_preprocess()

        df_scaled['pos'] = df_scaled['Pos'].apply(lambda x: any(d in x for d in demarcaciones))
        df_filtered = df_scaled[df_scaled['pos']].copy()

        # Cálculo de los scores escalados a 0–50
        df_filtered['Ataque Score'] = df_filtered[cols].apply(
            lambda x: self.multiply(list_pct_ataque, list_pct_defensa, x)[0] * 50, axis=1)

        df_filtered['Defensa Score'] = df_filtered[cols].apply(
            lambda x: self.multiply(list_pct_ataque, list_pct_defensa, x)[1] * 50, axis=1)

        df_filtered['Final Score'] = df_filtered['Ataque Score'] + df_filtered['Defensa Score']

        # Añadimos los valores reales de minutos jugados antes del merge
        df_filtered = df_filtered[['Jugador', 'Equipo', 'Pos', 'Edad', 'Ataque Score', 'Defensa Score', 'Final Score']]

        # Minutos jugados reales desde el DataFrame original
        minutos_reales = df_original[['Jugador', 'Equipo', 'Minutos jugados']]
        df_filtered = df_filtered.merge(minutos_reales, on=['Jugador', 'Equipo'])


        # Añadir las métricas normalizadas para análisis posterior
        df_scaled_metrics = df_scaled[['Jugador', 'Equipo'] + list(set(cols))]
        df_result = df_filtered.merge(df_scaled_metrics, on=['Jugador', 'Equipo'])

                # Filtro por minutos jugados
        min_cut = np.percentile(df_result['Minutos jugados'], percentile_min)
        df_result = df_result[df_result['Minutos jugados'] >= min_cut]

        # Redondear scores y métricas normalizadas
        cols_redondear = ['Ataque Score', 'Defensa Score', 'Final Score'] + list(set(cols))
        df_result[cols_redondear] = df_result[cols_redondear].round(2)

        df_result = df_result.sort_values(by='Final Score', ascending=is_ascending).reset_index(drop=True)
        return df_result.iloc[:n_top, :]


        df_result = df_result.sort_values(by='Final Score', ascending=is_ascending).reset_index(drop=True)
        return df_result.iloc[:n_top, :]



### Mejores Defensas Centrales

In [17]:
objeto = EstiloFutbolJugadores("/Users/pelayoquiros/Desktop/Proyect/Otras/Union_Valores_Final.xlsx")

best_centrales = objeto.algorithm(
    demarcaciones=['Defensa central'],
    cols=[
        'Pases_exito', 'Pases_progresivos', 'Pase_largo_exito', 
        'Regates_exito', 'Intercepciones', 'Despejes', 
        'Desafios_ganados', 'Arereo_ganado', 'Balon_recuperado', 'Tkl_ganados'
    ],
    percentile_min=80,
    n_top=60,
    is_ascending=False,
    list_pct_ataque=[0.25, 0.25, 0.25, 0.25],  
    list_pct_defensa=[0.16, 0.16, 0.16, 0.16, 0.16]       
)

best_centrales.head()


/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

,Jugador,Equipo,Pos,Edad,Ataque Score,Defensa Score,Final Score,Minutos jugados,Arereo_ganado,Tkl_ganados,Despejes,Regates_exito,Balon_recuperado,Pases_exito,Pase_largo_exito,Intercepciones,Pases_progresivos,Desafios_ganados
0,Virgil van Dijk,Liverpool,Defensa central,33,22.84,23.46,46.30,2520,0.67,0.23,0.72,0.03,0.56,0.84,0.46,0.81,0.50,0.18
1,Kim Min-jae,Bayern Munich,Defensa central,28,23.34,19.66,43.00,1915,0.55,0.28,0.33,0.02,0.82,0.87,0.29,0.58,0.68,0.18
2,Jan Paul van Hecke,Brighton,Defensa central,24,23.51,18.78,42.30,2150,0.48,0.39,0.49,0.03,0.65,0.63,0.50,0.38,0.71,0.35
3,Nathan Collins,Brentford,Defensa central,23,16.34,24.59,40.93,2430,0.65,0.30,0.81,0.06,0.77,0.55,0.31,0.45,0.40,0.39
4,Amir Rrahmani,Napoli,Defensa central,31,22.82,17.76,40.59,2340,0.61,0.34,0.50,0.02,0.57,0.73,0.56,0.30,0.52,0.24


### Mejores Laterales Izquierdos

In [18]:
objeto = EstiloFutbolJugadores("/Users/pelayoquiros/Desktop/Proyect/Otras/Union_Valores_Final.xlsx")

best_lateralizquierdo = objeto.algorithm(
    demarcaciones=['Lateral izquierdo'],
    cols=[
        'Pases_exito', 'Pases_progresivos', 'Pase_largo_exito', 
        'Regates_exito', 'Centros_area_exito', 'Intercepciones', 
        'Desafios_ganados', 'Arereo_ganado', 'Balon_recuperado', 'Despejes'
    ],
    percentile_min=80,
    n_top=60,
    is_ascending=False,
    list_pct_ataque=[0.20, 0.20, 0.20, 0.20, 0.20],  # 5 métricas de ataque
    list_pct_defensa=[0.2, 0.2, 0.2, 0.2, 0.2]       # 5 métricas de defensa
)

best_lateralizquierdo.head()

/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

,Jugador,Equipo,Pos,Edad,Ataque Score,Defensa Score,Final Score,Minutos jugados,Arereo_ganado,Despejes,Regates_exito,Balon_recuperado,Pases_exito,Pase_largo_exito,Intercepciones,Centros_area_exito,Pases_progresivos,Desafios_ganados
0,Antonee Robinson,Fulham,Lateral izquierdo,27,25.51,29.40,54.91,2359,0.27,0.46,0.30,0.70,0.53,0.32,0.89,1.00,0.40,0.63
1,Maximilian Mittelstädt,Stuttgart,Lateral izquierdo,27,22.97,19.52,42.48,1609,0.12,0.30,0.13,0.47,0.46,0.29,0.43,0.92,0.48,0.63
2,Lewis Hall,Newcastle Utd,Lateral izquierdo,20,16.96,24.84,41.80,2189,0.23,0.38,0.12,0.75,0.55,0.32,0.34,0.20,0.50,0.78
3,Johan Mojica,Mallorca,Lateral izquierdo,32,22.74,18.99,41.74,1931,0.05,0.29,0.27,0.79,0.34,0.31,0.45,0.96,0.39,0.31
4,Milos Kerkez,Bournemouth,Lateral izquierdo,21,18.69,22.86,41.56,2357,0.13,0.42,0.17,0.79,0.39,0.18,0.62,0.64,0.49,0.33


### Mejores Laterales Derechos

In [19]:
objeto = EstiloFutbolJugadores("/Users/pelayoquiros/Desktop/Proyect/Otras/Union_Valores_Final.xlsx")

best_lateralderecho = objeto.algorithm(
    demarcaciones=['Lateral derecho'],
    cols=[
        'Pases_exito', 'Pases_progresivos', 'Pase_largo_exito', 
        'Regates_exito', 'Centros_area_exito', 'Intercepciones', 
        'Desafios_ganados', 'Arereo_ganado', 'Balon_recuperado', 'Despejes'
    ],
    percentile_min=80,
    n_top=60,
    is_ascending=False,
    list_pct_ataque=[0.20, 0.20, 0.20, 0.20, 0.20],  # 5 métricas de ataque
    list_pct_defensa=[0.2, 0.2, 0.2, 0.2, 0.2]       # 5 métricas de defensa
)

best_lateralderecho.head()

/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

,Jugador,Equipo,Pos,Edad,Ataque Score,Defensa Score,Final Score,Minutos jugados,Arereo_ganado,Despejes,Regates_exito,Balon_recuperado,Pases_exito,Pase_largo_exito,Intercepciones,Centros_area_exito,Pases_progresivos,Desafios_ganados
0,Trent Alexander-Arnold,Liverpool,Lateral derecho,26,26.79,23.98,50.77,2057,0.02,0.26,0.10,0.74,0.52,0.53,0.53,0.76,0.77,0.84
1,Pedro Porro,Tottenham,Lateral derecho,25,22.71,26.12,48.84,2194,0.08,0.37,0.11,0.81,0.49,0.35,0.58,0.80,0.52,0.76
2,Aaron Wan-Bissaka,West Ham,Lateral derecho,27,18.69,24.85,43.54,2254,0.14,0.45,0.48,0.82,0.42,0.12,0.83,0.44,0.41,0.25
3,Guela Doué,Strasbourg,Lateral derecho,22,15.77,26.50,42.27,1906,0.28,0.54,0.16,0.77,0.50,0.25,0.53,0.20,0.47,0.53
4,Dodô,Fiorentina,Lateral derecho,26,20.82,21.45,42.26,2226,0.08,0.25,0.44,0.86,0.41,0.22,0.53,0.60,0.41,0.43


### Mejores Mediocentros

In [22]:
best_mediocentros = objeto.algorithm(
    demarcaciones=['Mediocentro'],
    cols=[
        'Pases_exito', 'Pases_progresivos', 'Regates_exito', 
        'xA', 'Pases_clave', 'Pases_exito_area', 
        'Toques_balon_juego', 'Arereo_ganado', 'Balon_recuperado', 'Intercepciones'
    ],
    percentile_min=80,
    n_top=60,
    is_ascending=False,
    list_pct_ataque=[0.14, 0.14, 0.14, 0.14, 0.14, 0.14,0.14],  
    list_pct_defensa=[0.33, 0.33, 0.33]       
)

best_mediocentros.head()

/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

,Jugador,Equipo,Pos,Edad,Ataque Score,Defensa Score,Final Score,Minutos jugados,Arereo_ganado,Regates_exito,Pases_exito_area,Balon_recuperado,Pases_exito,Toques_balon_juego,Intercepciones,Pases_clave,Pases_progresivos,xA
0,Bruno Fernandes,Manchester Utd,Mediocentro ofensivo,30,31.05,22.88,53.93,2195,0.13,0.21,0.74,0.88,0.51,0.69,0.38,0.81,0.90,0.58
1,Pedri,Barcelona,Mediocentro,22,29.28,21.03,50.30,1869,0.02,0.30,0.63,0.99,0.56,0.63,0.26,0.59,0.83,0.65
2,Federico Valverde,Real Madrid,Mediocentro,26,20.59,28.55,49.14,2099,0.15,0.13,0.30,0.75,0.59,0.64,0.83,0.40,0.63,0.25
3,Youri Tielemans,Aston Villa,Mediocentro,27,27.64,21.22,48.86,2401,0.13,0.30,0.41,0.76,0.58,0.70,0.40,0.66,0.75,0.54
4,Éderson,Atalanta,Mediocentro,25,21.44,26.98,48.41,2070,0.26,0.16,0.44,0.85,0.53,0.61,0.53,0.30,0.61,0.41


### Mejores Mediocentros Ofensivos

In [27]:
objeto = EstiloFutbolJugadores("/Users/pelayoquiros/Desktop/Proyect/Otras/Union_Valores_Final.xlsx")

best_mediocentrosofensivo = objeto.algorithm(
    demarcaciones=['Mediocentro ofensivo'],
    cols=[
        'Pases_exito', 'Pases_progresivos', 'Regates_exito', 
        'Gls/90', 'Ast/90', 'Toques_area_penal_att', 
        'Pases_progresivos_recibidos', 'Arereo_ganado', 'Balon_recuperado', 'Intercepciones'
    ],
    percentile_min=80,
    n_top=60,
    is_ascending=False,
    list_pct_ataque=[0.14, 0.14, 0.14, 0.14, 0.14, 0.14,0.14],  
    list_pct_defensa=[0.33, 0.33, 0.33] 
)

best_mediocentrosofensivo.head()

/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

,Jugador,Equipo,Pos,Edad,Ataque Score,Defensa Score,Final Score,Minutos jugados,Arereo_ganado,Regates_exito,Pases_progresivos_recibidos,Balon_recuperado,Pases_exito,Toques_area_penal_att,Intercepciones,Gls/90,Pases_progresivos,Ast/90
0,Bruno Fernandes,Manchester Utd,Mediocentro ofensivo,30,15.32,22.88,38.20,2195,0.13,0.21,0.31,0.88,0.51,0.24,0.38,0.02,0.90,0.00
1,Romano Schmid,Werder Bremen,Mediocentro ofensivo,25,15.27,18.13,33.40,1958,0.14,0.30,0.41,0.75,0.37,0.28,0.21,0.00,0.81,0.00
2,Florian Wirtz,Leverkusen,Mediocentro ofensivo,21,20.59,12.68,33.27,1781,0.05,0.81,0.69,0.57,0.45,0.42,0.15,0.03,0.53,0.01
3,Adrien Thomasson,Lens,Mediocentro ofensivo,31,10.68,22.09,32.77,1858,0.11,0.18,0.26,0.95,0.38,0.27,0.28,0.01,0.42,0.00
4,Himad Abdelli,Angers,Mediocentro ofensivo,25,13.42,18.89,32.31,1952,0.08,0.46,0.26,0.77,0.42,0.15,0.30,0.02,0.60,0.00


### Mejores Pivotes

In [28]:
objeto = EstiloFutbolJugadores("/Users/pelayoquiros/Desktop/Proyect/Otras/Union_Valores_Final.xlsx")

best_pivote = objeto.algorithm(
    demarcaciones=['Pivote'],
    cols=[
        'Pases_exito', 'Pases_progresivos', 'Pase_largo_exito', 
        'Regates_exito', 'Intercepciones', 'Despejes', 
        'Desafios_ganados', 'Arereo_ganado', 'Balon_recuperado', 'Tkl_ganados'
    ],
    percentile_min=80,
    n_top=60,
    is_ascending=False,
    list_pct_ataque=[0.25, 0.25, 0.25, 0.25],  
    list_pct_defensa=[0.16, 0.16, 0.16, 0.16, 0.16]       
)

best_pivote.head()

/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

,Jugador,Equipo,Pos,Edad,Ataque Score,Defensa Score,Final Score,Minutos jugados,Arereo_ganado,Tkl_ganados,Despejes,Regates_exito,Balon_recuperado,Pases_exito,Pase_largo_exito,Intercepciones,Pases_progresivos,Desafios_ganados
0,Joshua Kimmich,Bayern Munich,Pivote,30,35.12,14.45,49.56,2022,0.07,0.38,0.10,0.10,0.87,1.00,0.71,0.30,1.00,0.47
1,Moisés Caicedo,Chelsea,Pivote,23,21.16,22.57,43.72,2361,0.18,0.75,0.19,0.16,1.00,0.61,0.36,0.68,0.56,0.76
2,Granit Xhaka,Leverkusen,Pivote,32,28.95,12.65,41.60,2060,0.18,0.33,0.15,0.07,0.74,0.77,0.49,0.15,0.99,0.35
3,Ryan Gravenberch,Liverpool,Pivote,22,19.08,21.72,40.81,2391,0.14,0.44,0.25,0.28,0.92,0.55,0.16,0.92,0.55,0.49
4,Benjamin André,Lille,Pivote,34,17.77,22.15,39.93,1792,0.54,0.39,0.19,0.06,0.81,0.45,0.31,0.60,0.61,0.63


### Mejores Extremos derechos

In [29]:
objeto = EstiloFutbolJugadores("/Users/pelayoquiros/Desktop/Proyect/Otras/Union_Valores_Final.xlsx")

best_extremoderecho = objeto.algorithm(
    demarcaciones=['Extremo derecho'],
    cols=[
        'Pases_exito', 'Centros_area_exito', 'Regates_exito', 
        'Gls/90', 'Ast/90', 'Toques_area_penal_att', 
        'Pases_progresivos_recibidos', 'Arereo_ganado', 'Balon_recuperado', 'Intercepciones'
    ],
    percentile_min=80,
    n_top=60,
    is_ascending=False,
    list_pct_ataque=[0.14, 0.14, 0.14, 0.14, 0.14, 0.14,0.14],  
    list_pct_defensa=[0.33, 0.33, 0.33] 
)

best_extremoderecho.head()

/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

,Jugador,Equipo,Pos,Edad,Ataque Score,Defensa Score,Final Score,Minutos jugados,Arereo_ganado,Regates_exito,Pases_progresivos_recibidos,Balon_recuperado,Pases_exito,Toques_area_penal_att,Intercepciones,Centros_area_exito,Gls/90,Ast/90
0,Antoine Semenyo,Bournemouth,Extremo derecho,25,16.18,20.11,36.29,2286,0.36,0.63,0.62,0.74,0.24,0.56,0.11,0.24,0.02,0.00
1,Bryan Mbeumo,Brentford,Extremo derecho,25,16.47,18.59,35.06,2424,0.19,0.38,0.76,0.71,0.27,0.41,0.23,0.48,0.04,0.00
2,Mohamed Salah,Liverpool,Extremo derecho,32,23.76,10.32,34.08,2478,0.05,0.56,1.00,0.50,0.27,1.00,0.08,0.48,0.07,0.01
3,Junya Ito,Reims,Extremo derecho,31,17.56,15.74,33.30,2008,0.10,0.44,0.61,0.63,0.27,0.25,0.23,0.92,0.01,0.00
4,Lamine Yamal,Barcelona,Extremo derecho,17,20.30,10.90,31.20,1786,0.00,1.00,0.72,0.47,0.25,0.50,0.19,0.40,0.02,0.01


### Mejores Extremos Izquierdos

In [30]:
objeto = EstiloFutbolJugadores("/Users/pelayoquiros/Desktop/Proyect/Otras/Union_Valores_Final.xlsx")

best_extremoizquierdo = objeto.algorithm(
    demarcaciones=['Extremo izquierdo'],
    cols=[
        'Pases_exito', 'Centros_area_exito', 'Regates_exito', 
        'Gls/90', 'Ast/90', 'Toques_area_penal_att', 
        'Pases_progresivos_recibidos', 'Arereo_ganado', 'Balon_recuperado', 'Intercepciones'
    ],
    percentile_min=80,
    n_top=60,
    is_ascending=False,
    list_pct_ataque=[0.14, 0.14, 0.14, 0.14, 0.14, 0.14,0.14],  
    list_pct_defensa=[0.33, 0.33, 0.33] 
)

best_extremoizquierdo.head()

/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

,Jugador,Equipo,Pos,Edad,Ataque Score,Defensa Score,Final Score,Minutos jugados,Arereo_ganado,Regates_exito,Pases_progresivos_recibidos,Balon_recuperado,Pases_exito,Toques_area_penal_att,Intercepciones,Centros_area_exito,Gls/90,Ast/90
0,Diego Moreira,Strasbourg,Extremo izquierdo,20,11.70,21.91,33.61,1799,0.19,0.40,0.46,0.67,0.29,0.20,0.47,0.32,0.00,0.0
1,Mikkel Damsgaard,Brentford,Extremo izquierdo,24,12.91,20.16,33.07,2042,0.08,0.45,0.28,0.74,0.34,0.21,0.40,0.56,0.01,0.0
2,Iliman Ndiaye,Everton,Extremo izquierdo,24,11.11,20.84,31.95,1877,0.03,0.65,0.27,0.91,0.19,0.24,0.32,0.20,0.02,0.0
3,Kaoru Mitoma,Brighton,Extremo izquierdo,27,15.91,14.17,30.08,2148,0.17,0.48,0.71,0.47,0.26,0.52,0.23,0.28,0.02,0.0
4,Luis Rioja,Valencia,Extremo izquierdo,31,13.37,15.76,29.13,1979,0.07,0.34,0.40,0.49,0.23,0.16,0.40,0.76,0.02,0.0


### Mejores Delanteros Centros

In [32]:
objeto = EstiloFutbolJugadores("/Users/pelayoquiros/Desktop/Proyect/Otras/Union_Valores_Final.xlsx")

best_delanteros = objeto.algorithm(
    demarcaciones=['Delantero centro'],
    cols=[
        'Disparos/90', 'xG/90', 'Regates_exito', 
        'Gls/90', 'Ast/90', 'Toques_area_penal_att', 
        'Pases_progresivos_recibidos', 'Arereo_ganado', 'Balon_recuperado', 'Intercepciones'
    ],
    percentile_min=80,
    n_top=60,
    is_ascending=False,
    list_pct_ataque=[0.14, 0.14, 0.14, 0.14, 0.14, 0.14,0.14],  
    list_pct_defensa=[0.33, 0.33, 0.33] 
)

best_delanteros.head()

/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_ataque = sum([p * cols[i] for i, p in enumerate(pcts_ataque)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value_defensa = sum([p * cols[i + len(pcts_ataque)] for i, p in enumerate(pcts_defensa)])
/var/folders/52/z3j8tkc95y3_f3dbcs3q6vpc0000gn/T/ipykernel_981/2580642723.py:41: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

,Jugador,Equipo,Pos,Edad,Ataque Score,Defensa Score,Final Score,Minutos jugados,Arereo_ganado,xG/90,Regates_exito,Pases_progresivos_recibidos,Balon_recuperado,Toques_area_penal_att,Intercepciones,Gls/90,Disparos/90,Ast/90
0,Matheus Cunha,Wolves,Delantero centro,25,9.91,17.65,27.55,2143,0.08,0.01,0.55,0.40,0.69,0.40,0.30,0.04,0.02,0.0
1,Tim Kleindienst,Gladbach,Delantero centro,29,6.32,19.54,25.86,1933,0.75,0.01,0.11,0.30,0.28,0.41,0.15,0.05,0.02,0.0
2,Philipp Hofmann,Bochum,Delantero centro,31,4.32,19.52,23.84,1607,0.86,0.00,0.10,0.23,0.27,0.26,0.06,0.01,0.01,0.0
3,Evann Guessand,Nice,Delantero centro,23,8.21,14.37,22.58,1955,0.17,0.01,0.39,0.36,0.48,0.37,0.23,0.03,0.01,0.0
4,Nikola Krstović,Lecce,Delantero centro,24,6.26,15.85,22.11,2147,0.51,0.01,0.16,0.29,0.34,0.39,0.11,0.02,0.02,0.0
